### 載入MNIST手寫數字數據集

In [1]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
import tensorflow as tf

In [28]:
def weight_variable(shape,name):
    return tf.Variable(tf.truncated_normal(shape = shape,stddev = 0.1),name = name)

def bias_variable(shape,name):
    return tf.Variable(tf.constant(0.1,shape = shape),name = name)

### 定義模型變數

In [24]:
n_input = 28
n_steps = 28
n_hidden = 128
n_classes = 10
X = tf.placeholder("float",[None,n_steps,n_input])
y_true = tf.placeholder("float",[None,10])

weights = {
    "w_fc" : weight_variable([n_hidden, n_classes], "w_fc")
}

biases = {
    "b_fc" : bias_variable([n_classes], "b_fc") 
}

In [25]:
x_transpose = tf.transpose(X,[1,0,2])
x_reshape = tf.reshape(x_transpose,[-1,n_input])
x_split = tf.split(x_reshape,n_steps,0)

### 定義模型

In [48]:
from tensorflow.contrib import rnn

basic_rnn_cell = rnn.BasicRNNCell(n_hidden)
outputs, states = rnn.static_rnn(basic_rnn_cell, x_split, dtype=tf.float32)

ValueError: Variable rnn/basic_rnn_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-32-0e945cb9e3ef>", line 4, in <module>
    outputs, states = rnn.static_rnn(basic_rnn_cell, x_split, dtype=tf.float32)
  File "/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2850, in run_ast_nodes
    if self.run_code(code, result):


In [33]:
h_fc = tf.matmul(outputs[-1],weights['w_fc'])+biases['b_fc']

In [43]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = h_fc,labels = y_true))
optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)

In [44]:
correct_prediction = tf.equal(tf.argmax(h_fc,1),tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [46]:
import numpy as np
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10000):
        batchx,batchy = mnist.train.next_batch(100)
        batchx = np.reshape(batchx, (100, n_steps, n_input))
        
        if step%1000 == 0:
            print("Step:{},Accuracy:{}".format(step,accuracy.eval(feed_dict = {X:batchx,y_true:batchy})))
        sess.run(optimizer,feed_dict = {X:batchx,y_true:batchy})
    
    print("Test Accuracy:{}".format(accuracy.eval(feed_dict = {X:np.reshape(mnist.test.images,[-1,28,28]),y_true:mnist.test.labels})))

Step:0,Accuracy:0.20999999344348907
Step:1000,Accuracy:0.4099999964237213
Step:2000,Accuracy:0.4699999988079071
Step:3000,Accuracy:0.12999999523162842
Step:4000,Accuracy:0.07000000029802322
Step:5000,Accuracy:0.07999999821186066
Step:6000,Accuracy:0.09000000357627869
Step:7000,Accuracy:0.10000000149011612
Step:8000,Accuracy:0.12999999523162842
Step:9000,Accuracy:0.07999999821186066
Test Accuracy:0.09799999743700027
